In [1]:
import pandas as pd 
import numpy as np
import os, sys 
import sqlite3
import plotly as py
import plotly.express as px
import plotly.graph_objs as go


In [7]:
pm_forms_data=pd.read_csv('Construction_Data_PM_Forms_All_Projects.csv')
pm_forms_data.head()

,Ref,Status,Location,Name,Created,Type,Status Changed,Open Actions,Total Actions,Association,OverDue,Images,Comments,Documents,Project,Report Forms Status,Report Forms Group
0,F145185.4,Opened,01 Daily Site Diary>Site Management>JPC Projec...,1328 CM-SM-FRM-001 Site Diary,15/09/2020,Site Management,15/09/2020,0,0,NaN,False,True,False,False,1328,Open,Site Management
1,F1.495500,Open / Ongoing Works,02 Daily Work Plan>Site Management>JPC Project...,SM-FRM-SUB-101 Daily Work Plan,15/09/2020,Subcontractor Inspections,15/09/2020,0,0,NaN,False,False,False,False,1328,Open,Subcontractor
2,F1.495499,Open / Ongoing Works,02 Daily Work Plan>Site Management>JPC Project...,SM-FRM-SUB-101 Daily Work Plan,15/09/2020,Subcontractor Inspections,15/09/2020,0,0,NaN,False,False,False,False,1328,Open,Subcontractor
3,F1.495498,Open / Ongoing Works,02 Daily Work Plan>Site Management>JPC Project...,SM-FRM-SUB-101 Daily Work Plan,15/09/2020,Subcontractor Inspections,15/09/2020,0,0,NaN,False,False,False,False,1328,Open,Subcontractor
4,F1.495496,Open / Ongoing Works,02 Daily Work Plan>Site Management>JPC Project...,SM-FRM-SUB-101 Daily Work Plan,15/09/2020,Subcontractor Inspections,15/09/2020,0,0,NaN,False,False,False,False,1328,Open,Subcontractor


In [8]:
pm_tasks_data=pd.read_csv('Construction_Data_PM_Tasks_All_Projects.csv')
pm_tasks_data.head()

,Ref,Status,Location,Description,Created,Target,Type,To Package,Status Changed,Association,OverDue,Images,Comments,Documents,Priority,Cause,project,Report Status,Task Group
0,T1.23963030,Open,JPC Project Management>EHS Management>01 Inspe...,task raised in incorrect location of this form...,14/09/2020,NaN,Safety Notice (Amber) - General Issue,Main Contractor,14/09/2020,FormAnswer,False,NaN,NaN,NaN,Behavioural Failure,JPC - Safety - Documentation,1328,Open,Safety
1,T116412.200,Closed,QC & BC(A)R>ITP 02 Architectural & M&E Service...,Metsec,14/09/2020,NaN,JPC - Progress Photo,Ceilings & Partitions,14/09/2020,NaN,False,True,False,False,NaN,NaN,1328,Closed,Site Management
2,T141663.27,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,Good clear exclusion zones and access through ...,14/09/2020,NaN,Safety Notice (Green) - Good Observation,Main Contractor,14/09/2020,FormAnswer,False,True,False,False,NaN,JPC - Safety - Access,1328,Closed,Safety
3,T116412.199,Closed,QC & BC(A)R>ITP 02 Architectural & M&E Service...,RC walls,14/09/2020,NaN,JPC - Progress Photo,Precast Concrete,14/09/2020,NaN,False,True,False,False,NaN,NaN,1328,Closed,Site Management
4,T141663.26,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,"block 02 working level has good housekeeping, ...",14/09/2020,NaN,Safety Notice (Green) - Good Observation,Precast Concrete,14/09/2020,FormAnswer,False,True,False,False,NaN,JPC - Safety - House Keeping,1328,Closed,Safety


In [9]:
pm_tasks_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12424 entries, 0 to 12423
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ref             12424 non-null  object 
 1   Status          12424 non-null  object 
 2   Location        12424 non-null  object 
 3   Description     12424 non-null  object 
 4   Created         12424 non-null  object 
 5   Target          2568 non-null   float64
 6   Type            12424 non-null  object 
 7   To Package      11382 non-null  object 
 8   Status Changed  12424 non-null  object 
 9   Association     9483 non-null   object 
 10  OverDue         12424 non-null  bool   
 11  Images          12272 non-null  object 
 12  Comments        11902 non-null  object 
 13  Documents       11780 non-null  object 
 14  Priority        2366 non-null   object 
 15  Cause           9683 non-null   object 
 16  project         12424 non-null  int64  
 17  Report Status   12424 non-null 

In [11]:
pm_tasks_data.describe()

,Target,project
count,2568.000000,12424.000000
mean,43968.516355,1332.585480
std,105.261518,5.213831
min,43590.000000,1328.000000
25%,43923.750000,1328.000000
50%,44000.000000,1330.000000
75%,44041.000000,1338.000000
max,44106.000000,1345.000000


In [12]:
pm_tasks_data.shape


(12424, 19)

In [13]:
pm_tasks_data['Created']=pd.to_datetime(pm_tasks_data['Created'], format='%d/%m/%Y')
pm_tasks_data.head(5)

,Ref,Status,Location,Description,Created,Target,Type,To Package,Status Changed,Association,OverDue,Images,Comments,Documents,Priority,Cause,project,Report Status,Task Group
0,T1.23963030,Open,JPC Project Management>EHS Management>01 Inspe...,task raised in incorrect location of this form...,2020-09-14,NaN,Safety Notice (Amber) - General Issue,Main Contractor,14/09/2020,FormAnswer,False,NaN,NaN,NaN,Behavioural Failure,JPC - Safety - Documentation,1328,Open,Safety
1,T116412.200,Closed,QC & BC(A)R>ITP 02 Architectural & M&E Service...,Metsec,2020-09-14,NaN,JPC - Progress Photo,Ceilings & Partitions,14/09/2020,NaN,False,True,False,False,NaN,NaN,1328,Closed,Site Management
2,T141663.27,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,Good clear exclusion zones and access through ...,2020-09-14,NaN,Safety Notice (Green) - Good Observation,Main Contractor,14/09/2020,FormAnswer,False,True,False,False,NaN,JPC - Safety - Access,1328,Closed,Safety
3,T116412.199,Closed,QC & BC(A)R>ITP 02 Architectural & M&E Service...,RC walls,2020-09-14,NaN,JPC - Progress Photo,Precast Concrete,14/09/2020,NaN,False,True,False,False,NaN,NaN,1328,Closed,Site Management
4,T141663.26,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,"block 02 working level has good housekeeping, ...",2020-09-14,NaN,Safety Notice (Green) - Good Observation,Precast Concrete,14/09/2020,FormAnswer,False,True,False,False,NaN,JPC - Safety - House Keeping,1328,Closed,Safety


In [14]:
pm_tasks_data['Status Changed']=pd.to_datetime(pm_tasks_data['Status Changed'], format='%d/%m/%Y')
pm_tasks_data.head(5)

,Ref,Status,Location,Description,Created,Target,Type,To Package,Status Changed,Association,OverDue,Images,Comments,Documents,Priority,Cause,project,Report Status,Task Group
0,T1.23963030,Open,JPC Project Management>EHS Management>01 Inspe...,task raised in incorrect location of this form...,2020-09-14,NaN,Safety Notice (Amber) - General Issue,Main Contractor,2020-09-14,FormAnswer,False,NaN,NaN,NaN,Behavioural Failure,JPC - Safety - Documentation,1328,Open,Safety
1,T116412.200,Closed,QC & BC(A)R>ITP 02 Architectural & M&E Service...,Metsec,2020-09-14,NaN,JPC - Progress Photo,Ceilings & Partitions,2020-09-14,NaN,False,True,False,False,NaN,NaN,1328,Closed,Site Management
2,T141663.27,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,Good clear exclusion zones and access through ...,2020-09-14,NaN,Safety Notice (Green) - Good Observation,Main Contractor,2020-09-14,FormAnswer,False,True,False,False,NaN,JPC - Safety - Access,1328,Closed,Safety
3,T116412.199,Closed,QC & BC(A)R>ITP 02 Architectural & M&E Service...,RC walls,2020-09-14,NaN,JPC - Progress Photo,Precast Concrete,2020-09-14,NaN,False,True,False,False,NaN,NaN,1328,Closed,Site Management
4,T141663.26,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,"block 02 working level has good housekeeping, ...",2020-09-14,NaN,Safety Notice (Green) - Good Observation,Precast Concrete,2020-09-14,FormAnswer,False,True,False,False,NaN,JPC - Safety - House Keeping,1328,Closed,Safety


In [15]:
pm_tasks_data['Task Group'].unique()


array(['Safety', 'Site Management', 'Quality', 'Design Team', nan],
      dtype=object)

In [16]:
print('Number of Projects: ', pm_tasks_data['project'].nunique())
print('Number of Trade Packages: ', pm_tasks_data['To Package'].nunique())
print('Number of Task Groups: ', pm_tasks_data['Task Group'].nunique())

Number of Projects:  8
Number of Trade Packages:  105
Number of Task Groups:  4


In [17]:
missing_values_count = pm_tasks_data.isnull().sum()
missing_values_count[:]

Ref                   0
Status                0
Location              0
Description           0
Created               0
Target             9856
Type                  0
To Package         1042
Status Changed        0
Association        2941
OverDue               0
Images              152
Comments            522
Documents           644
Priority          10058
Cause              2741
project               0
Report Status         0
Task Group           50
dtype: int64

In [18]:
print(pm_tasks_data.size)

236056


In [22]:
# how many total missing values do we have?
total_cells = np.product(pm_tasks_data.shape)
total_missing = missing_values_count.sum()
print(total_missing)
((total_missing/total_cells) * 100).round(2)

28006


11.86

In [23]:
pm_tasks_data['Priority'].value_counts()

Behavioural Failure                                                 951
System Failure                                                      787
Medium                                                              221
High                                                                214
Best Practice                                                        83
System Failure - Deviation from RAMS / Manufacturer Instructions     31
Low                                                                  29
.                                                                    27
2 Week Look Ahead                                                    13
Low (resolve within 2 weeks)                                          3
Medium (resolve within 5 days)                                        2
High (resolve within 48 hours)                                        2
1 Week Look Ahead                                                     2
1 Month Look Ahead                                              

In [24]:
pm_tasks_data['Priority'].isnull()


0        False
1         True
2         True
3         True
4         True
         ...  
12419    False
12420     True
12421    False
12422     True
12423     True
Name: Priority, Length: 12424, dtype: bool

In [25]:
pm_tasks_data.loc[(pm_tasks_data['Priority'].isnull()) & (pm_tasks_data['Task Group']=='Safety'), 'Priority']='Behavioural Failure'

In [26]:
pm_tasks_data.loc[pm_tasks_data['Task Group'] == 'Safety']

,Ref,Status,Location,Description,Created,Target,Type,To Package,Status Changed,Association,OverDue,Images,Comments,Documents,Priority,Cause,project,Report Status,Task Group
0,T1.23963030,Open,JPC Project Management>EHS Management>01 Inspe...,task raised in incorrect location of this form...,2020-09-14,NaN,Safety Notice (Amber) - General Issue,Main Contractor,2020-09-14,FormAnswer,False,NaN,NaN,NaN,Behavioural Failure,JPC - Safety - Documentation,1328,Open,Safety
2,T141663.27,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,Good clear exclusion zones and access through ...,2020-09-14,NaN,Safety Notice (Green) - Good Observation,Main Contractor,2020-09-14,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - Access,1328,Closed,Safety
4,T141663.26,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,"block 02 working level has good housekeeping, ...",2020-09-14,NaN,Safety Notice (Green) - Good Observation,Precast Concrete,2020-09-14,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - House Keeping,1328,Closed,Safety
6,T141663.25,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,operaiver using hand sanitizer points onsite. ...,2020-09-14,NaN,Safety Notice (Green) - Good Observation,Ceilings & Partitions,2020-09-14,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - Welfare Facilities,1328,Closed,Safety
8,T141663.24,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,number of operative fully pee compliant,2020-09-14,NaN,Safety Notice (Green) - Good Observation,Main Contractor,2020-09-14,FormAnswer,False,False,False,False,Behavioural Failure,JPC - Safety - PPE,1328,Closed,Safety
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12419,T122039.5,Closed,1345 - DUB062 Project,Extend temp road and get IAC to install requir...,2020-03-11,NaN,Safety Notice (Amber) - General Issue,Main Contractor,2020-03-23,FormAnswer,False,True,True,False,System Failure,JPC - Safety - Access,1345,Closed,Safety
12420,T122039.3,EHS Good Observation,1345 - DUB062 Project,Canteen in condition,2020-03-11,NaN,Safety Notice (Green) - Good Observation,Main Contractor,2020-03-11,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - Welfare Facilities,1345,Closed,Safety
12421,T122039.1,Closed,1345 - DUB062 Project,Complete drying room set up,2020-03-11,NaN,Safety Notice (Amber) - General Issue,Main Contractor,2020-03-19,FormAnswer,False,True,True,False,System Failure,JPC - Safety - Welfare Facilities,1345,Closed,Safety
12422,T120669.2,Closed,1345 - DUB062 Project,ga1 not available for some plant on site. requ...,2020-02-20,NaN,Safety Notice (Amber) - General Issue,Main Contractor,2020-02-28,FormAnswer,False,False,True,True,Behavioural Failure,JPC - Safety - Plant & Equipment,1345,Closed,Safety


In [27]:
#the remaining rows can be filled with Other.
pm_tasks_data['Priority'] = pm_tasks_data['Priority'].fillna('Other')
pm_tasks_data['Priority']

0        Behavioural Failure
1                      Other
2        Behavioural Failure
3                      Other
4        Behavioural Failure
                ...         
12419         System Failure
12420    Behavioural Failure
12421         System Failure
12422    Behavioural Failure
12423    Behavioural Failure
Name: Priority, Length: 12424, dtype: object

In [28]:
pm_tasks_data['Association'].value_counts()

FormAnswer       9281
ForwardedFrom     193
ForwardedTo         9
Name: Association, dtype: int64

In [29]:
pm_tasks_data['Association'].unique()

array(['FormAnswer', nan, 'ForwardedFrom', 'ForwardedTo'], dtype=object)

In [30]:
pm_tasks_data['Cause'].value_counts().head(5)


JPC - Safety - House Keeping      2039
JPC - Safety - Access             1577
JPC - Safety - Exclusion Zones     734
JPC - Quality - Workmanship        631
JPC - Safety - Storage             555
Name: Cause, dtype: int64

In [31]:
pm_tasks_data['Cause'] = pm_tasks_data['Cause'].fillna('JPC - Safety - House Keeping ')
pm_tasks_data['Cause']

0             JPC - Safety - Documentation
1            JPC - Safety - House Keeping 
2                    JPC - Safety - Access
3            JPC - Safety - House Keeping 
4             JPC - Safety - House Keeping
                       ...                
12419                JPC - Safety - Access
12420    JPC - Safety - Welfare Facilities
12421    JPC - Safety - Welfare Facilities
12422     JPC - Safety - Plant & Equipment
12423    JPC - Safety - Welfare Facilities
Name: Cause, Length: 12424, dtype: object

In [32]:
pm_tasks_data[['Images','Comments','Documents']] = pm_tasks_data[['Images','Comments','Documents']].fillna('N/A')
pm_tasks_data[['Images','Comments','Documents']]


,Images,Comments,Documents
0,N/A,N/A,N/A
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
...,...,...,...
12419,True,True,False
12420,True,False,False
12421,True,True,False
12422,False,True,True


In [33]:
pm_tasks_data['Task Group'].value_counts()

Safety             8884
Site Management    1420
Design Team        1139
Quality             931
Name: Task Group, dtype: int64

In [34]:
pm_tasks_data['Task Group'] = pm_tasks_data['Task Group'].fillna('Safety')
pm_tasks_data['Task Group']

0                 Safety
1        Site Management
2                 Safety
3        Site Management
4                 Safety
              ...       
12419             Safety
12420             Safety
12421             Safety
12422             Safety
12423             Safety
Name: Task Group, Length: 12424, dtype: object

In [35]:
# number of missing data points per column
missing_values_count = pm_tasks_data.isnull().sum()

# look at the # of missing points in each column
missing_values_count[:]

Ref                  0
Status               0
Location             0
Description          0
Created              0
Target            9856
Type                 0
To Package        1042
Status Changed       0
Association       2941
OverDue              0
Images               0
Comments             0
Documents            0
Priority             0
Cause                0
project              0
Report Status        0
Task Group           0
dtype: int64

In [36]:
pm_tasks_data['Target'].value_counts()

44011.0    75
44015.0    67
43998.0    52
43966.0    46
44070.0    43
           ..
43814.0     1
43792.0     1
43759.0     1
43760.0     1
43843.0     1
Name: Target, Length: 255, dtype: int64

In [37]:
pm_tasks_data['Target'].unique()

array([   nan, 44094., 44091., 44090., 44089., 44088., 44087., 44083.,
       44082., 44084., 44081., 44076., 44075., 44074., 44070., 44069.,
       44068., 44061., 44059., 44032., 44027., 44026., 44020., 44013.,
       44019., 44018., 44017., 44014., 44012., 44011., 44010., 44007.,
       43999., 44006., 44005., 43997., 44004., 44003., 43998., 43993.,
       43991., 43986., 43992., 43985., 43978., 43928., 43922., 43921.,
       43915., 43909., 43908., 43907., 43906., 43901., 43902., 43900.,
       43899., 43892., 43896., 43895., 43894., 43893., 43885., 43889.,
       43887., 43886., 43879., 43875., 43868., 43874., 43867., 43873.,
       43864., 43881., 43866., 43865., 43852., 43850., 43846., 43843.,
       43840., 43823., 43815., 43811., 43803., 43808., 43805., 43773.,
       43810., 43802., 43801., 43796., 43795., 43812., 43794., 43781.,
       43782., 43787., 43784., 43776., 43775., 43771., 43770., 43769.,
       43768., 43767., 43762., 43760., 43753., 43752., 43748., 43747.,
      

In [38]:
pm_tasks_data['Target'] = pm_tasks_data['Target'].fillna(44011.0)
pm_tasks_data['Target']


0        44011.0
1        44011.0
2        44011.0
3        44011.0
4        44011.0
          ...   
12419    44011.0
12420    44011.0
12421    44011.0
12422    44011.0
12423    44011.0
Name: Target, Length: 12424, dtype: float64

In [39]:
pm_tasks_data['Association'].value_counts()

FormAnswer       9281
ForwardedFrom     193
ForwardedTo         9
Name: Association, dtype: int64

In [40]:
pm_tasks_data['Association'] = pm_tasks_data['Association'].fillna('FormAnswer')
pm_tasks_data['Association']

0        FormAnswer
1        FormAnswer
2        FormAnswer
3        FormAnswer
4        FormAnswer
            ...    
12419    FormAnswer
12420    FormAnswer
12421    FormAnswer
12422    FormAnswer
12423    FormAnswer
Name: Association, Length: 12424, dtype: object

In [41]:
pm_tasks_data['Cause'].value_counts().head(5)

JPC - Safety - House Keeping      2741
JPC - Safety - House Keeping      2039
JPC - Safety - Access             1577
JPC - Safety - Exclusion Zones     734
JPC - Quality - Workmanship        631
Name: Cause, dtype: int64

In [42]:
pm_tasks_data['Cause'] = pm_tasks_data['Cause'].fillna('JPC - Safety - House Keeping ')
pm_tasks_data['Cause']

0             JPC - Safety - Documentation
1            JPC - Safety - House Keeping 
2                    JPC - Safety - Access
3            JPC - Safety - House Keeping 
4             JPC - Safety - House Keeping
                       ...                
12419                JPC - Safety - Access
12420    JPC - Safety - Welfare Facilities
12421    JPC - Safety - Welfare Facilities
12422     JPC - Safety - Plant & Equipment
12423    JPC - Safety - Welfare Facilities
Name: Cause, Length: 12424, dtype: object

In [43]:
pm_tasks_data['To Package'].value_counts()

Main Contractor                   3461
Construction Management            781
Concrete - Formwork                765
Internal Partitions & Ceilings     595
Groundworks & Drainage             500
                                  ... 
Pool Specialist                      1
External Render                      1
Architectural Steelwork              1
Civil Engineer                       1
Electrical Subcontractor             1
Name: To Package, Length: 105, dtype: int64

In [44]:
pm_tasks_data['To Package'] = pm_tasks_data['To Package'].fillna('Design')
pm_tasks_data['To Package']

0              Main Contractor
1        Ceilings & Partitions
2              Main Contractor
3             Precast Concrete
4             Precast Concrete
                 ...          
12419          Main Contractor
12420          Main Contractor
12421          Main Contractor
12422          Main Contractor
12423          Main Contractor
Name: To Package, Length: 12424, dtype: object

In [45]:
pm_tasks_data[['Images','Comments','Documents']] = pm_tasks_data[['Images','Comments','Documents']].fillna('N/A')
pm_tasks_data[['Images','Comments','Documents']]


,Images,Comments,Documents
0,N/A,N/A,N/A
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
...,...,...,...
12419,True,True,False
12420,True,False,False
12421,True,True,False
12422,False,True,True


In [46]:
pm_tasks_data['Task Group'].value_counts()

Safety             8934
Site Management    1420
Design Team        1139
Quality             931
Name: Task Group, dtype: int64

In [47]:
pm_tasks_data['Task Group'] = pm_tasks_data['Task Group'].fillna('Safety')
pm_tasks_data['Task Group']

0                 Safety
1        Site Management
2                 Safety
3        Site Management
4                 Safety
              ...       
12419             Safety
12420             Safety
12421             Safety
12422             Safety
12423             Safety
Name: Task Group, Length: 12424, dtype: object

In [48]:
missing_values_count = pm_tasks_data.isnull().sum()

# look at the # of missing points in each column
missing_values_count[:]

Ref               0
Status            0
Location          0
Description       0
Created           0
Target            0
Type              0
To Package        0
Status Changed    0
Association       0
OverDue           0
Images            0
Comments          0
Documents         0
Priority          0
Cause             0
project           0
Report Status     0
Task Group        0
dtype: int64

In [49]:
#non-numerical data columns 
non_numeric_data = list(pm_tasks_data.select_dtypes(exclude=('int', 'float')).columns)
print(f'Сolumns without numeric data: {", ".join(non_numeric_data)}.')
print(len(non_numeric_data))

Сolumns without numeric data: Ref, Status, Location, Description, Created, Type, To Package, Status Changed, Association, OverDue, Images, Comments, Documents, Priority, Cause, Report Status, Task Group.
17


In [50]:
print('Duplication Location:', 100*(len(pm_tasks_data['Location']) - len(pm_tasks_data['Location'].drop_duplicates()))/len(pm_tasks_data['Location']))


Duplication Location: 95.21893110109465


In [51]:
print('Duplication Status:', 100*(len(pm_tasks_data['Status']) - len(pm_tasks_data['Status'].drop_duplicates()))/len(pm_tasks_data['Status']))

Duplication Status: 99.87121699935608


In [52]:
print('Duplication Description:', 100*(len(pm_tasks_data['Description']) - len(pm_tasks_data['Description'].drop_duplicates()))/len(pm_tasks_data['Description']))

Duplication Description: 17.353509336767548


In [53]:
pm_tasks_data[pm_tasks_data.duplicated(['Description'], keep=False)]

,Ref,Status,Location,Description,Created,Target,Type,To Package,Status Changed,Association,OverDue,Images,Comments,Documents,Priority,Cause,project,Report Status,Task Group
11,T141663.21,Open,JPC Project Management>EHS Management>01 Inspe...,toe board missing,2020-09-14,44011.0,Safety Notice (Amber) - General Issue,Scaffolding,2020-09-14,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - Access,1328,Open,Safety
16,T141663.16,Open,JPC Project Management>EHS Management>01 Inspe...,no ladder tag,2020-09-14,44011.0,Safety Notice (Amber) - General Issue,Temporary Electrics,2020-09-14,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - WAH Equipment,1328,Open,Safety
17,T141663.15,Open,JPC Project Management>EHS Management>01 Inspe...,no ladder tag,2020-09-14,44011.0,Safety Notice (Amber) - General Issue,Precast Concrete,2020-09-14,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - Ladders,1328,Open,Safety
19,T141663.13,Open,JPC Project Management>EHS Management>01 Inspe...,drinking bottles from contractors not being di...,2020-09-14,44011.0,Safety Notice (Amber) - General Issue,Ceilings & Partitions,2020-09-14,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - House Keeping,1328,Open,Safety
20,T116412.196,Open / Ongoing Works,QC & BC(A)R>ITP 02 Architectural & M&E Service...,Flatpack duct not sealed laterally.,2020-09-14,44094.0,JPC - Quality Defect (compliance),Ceilings & Partitions,2020-09-14,FormAnswer,False,True,True,False,Other,JPC - Quality - Workmanship,1328,Open,Quality
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12288,T135787.7,Closed,JPC Project Management>EHS Inspections & Asses...,Housekeeping,2020-08-04,44011.0,Safety Notice (Amber) - General Issue,JPC Labour,2020-08-05,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Quality - Materials,1345,Closed,Safety
12304,T135786.33,Closed,JPC Project Management>EHS Inspections & Asses...,Housekeeping,2020-07-27,44011.0,Safety Notice (Amber) - General Issue,Hoarding,2020-08-05,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - House Keeping,1345,Closed,Safety
12356,T135787.3,Closed,JPC Project Management>EHS Inspections & Asses...,rutted ground outside temp compound to be made...,2020-07-09,44011.0,Safety Notice (Amber) - General Issue,Main Contractor,2020-07-23,FormAnswer,False,True,True,False,Behavioural Failure,JPC - Safety - Excavations,1345,Closed,Safety
12357,T135787.1,Closed,JPC Project Management>EHS Inspections & Asses...,rutted ground outside temp compound to be made...,2020-07-09,44011.0,Safety Notice (Amber) - General Issue,JPC Labour,2020-07-23,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - Access,1345,Closed,Safety


In [54]:
pm_tasks_data.head()

,Ref,Status,Location,Description,Created,Target,Type,To Package,Status Changed,Association,OverDue,Images,Comments,Documents,Priority,Cause,project,Report Status,Task Group
0,T1.23963030,Open,JPC Project Management>EHS Management>01 Inspe...,task raised in incorrect location of this form...,2020-09-14,44011.0,Safety Notice (Amber) - General Issue,Main Contractor,2020-09-14,FormAnswer,False,N/A,N/A,N/A,Behavioural Failure,JPC - Safety - Documentation,1328,Open,Safety
1,T116412.200,Closed,QC & BC(A)R>ITP 02 Architectural & M&E Service...,Metsec,2020-09-14,44011.0,JPC - Progress Photo,Ceilings & Partitions,2020-09-14,FormAnswer,False,True,False,False,Other,JPC - Safety - House Keeping,1328,Closed,Site Management
2,T141663.27,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,Good clear exclusion zones and access through ...,2020-09-14,44011.0,Safety Notice (Green) - Good Observation,Main Contractor,2020-09-14,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - Access,1328,Closed,Safety
3,T116412.199,Closed,QC & BC(A)R>ITP 02 Architectural & M&E Service...,RC walls,2020-09-14,44011.0,JPC - Progress Photo,Precast Concrete,2020-09-14,FormAnswer,False,True,False,False,Other,JPC - Safety - House Keeping,1328,Closed,Site Management
4,T141663.26,EHS Good Observation,JPC Project Management>EHS Management>01 Inspe...,"block 02 working level has good housekeeping, ...",2020-09-14,44011.0,Safety Notice (Green) - Good Observation,Precast Concrete,2020-09-14,FormAnswer,False,True,False,False,Behavioural Failure,JPC - Safety - House Keeping,1328,Closed,Safety


In [55]:
subset_project_tasks = pm_tasks_data[['Ref', 'Status', 'Location', 'Description', 'Created', 'Type', 'To Package', 'Status Changed', 'Priority', 'Cause', 'project', 'Report Status', 'Task Group']]
subset_project_tasks.dtypes

Ref                       object
Status                    object
Location                  object
Description               object
Created           datetime64[ns]
Type                      object
To Package                object
Status Changed    datetime64[ns]
Priority                  object
Cause                     object
project                    int64
Report Status             object
Task Group                object
dtype: object

In [56]:
subset_project_tasks.shape

(12424, 13)